In [4]:
%matplotlib inline
import itertools
from os.path import isfile
from glob import glob
import h5py
from matplotlib import pyplot as plt
import re
import sys
import random
import numpy as np
from skimage import io
from skimage.color import rgb2gray
import pandas as pd
from tqdm import trange, tqdm
from sklearn.metrics import precision_recall_curve
plt.style.use('ggplot')

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [8]:
image_paths = [
    'D:/School/hbo/sem4/OI/intelligentbeehive/examples/WDD_paper/GroundTruthData/GTRuns/wddGroundTruth/',
    'D:/School/hbo/sem4/OI/intelligentbeehive/examples/WDD_paper/GroundTruthData/GTRuns/20160814_1001_1/'
]

In [9]:
x1 = list(itertools.chain.from_iterable(map(lambda path: glob(path + "*/*/"), image_paths)))

In [10]:
x2 = np.zeros((len(x1),)).astype(int)
for i, x in enumerate(x1):
    x2[i] = len(glob(x + '/image_*.png'))

In [11]:
csv_path_16 = '/mnt/storage/ben/data/BeesBook/AdeferDanceDetection/20170620_1606_FE.csv'
gt_data_16 = pd.read_csv(csv_path_16, header=None, names=['key', 'gt_intern', 'gt'], index_col=0)
gt_data_16 = gt_data_16[~gt_data_16.index.duplicated(keep='first')]
gt_data_16.drop('gt_intern', axis=1, inplace=True)
gt_data_16.head()

FileNotFoundError: [Errno 2] File b'/mnt/storage/ben/data/BeesBook/AdeferDanceDetection/20170620_1606_FE.csv' does not exist: b'/mnt/storage/ben/data/BeesBook/AdeferDanceDetection/20170620_1606_FE.csv'

In [ ]:
csv_path_14 = '/mnt/storage/ben/data/BeesBook/AdeferDanceDetection/GT_20160816.csv'
gt_data_14 = pd.read_csv(csv_path_14, header=None, names=['key', 'gt'], index_col=0)
gt_data_14 = gt_data_14[~gt_data_14.index.duplicated(keep='first')]
gt_data_14.head()

In [ ]:
gt_data = pd.concat((gt_data_14, gt_data_16), axis=0)
gt_data.head()

In [ ]:
X1 = []
X2 = []
Y = []
for (i, path), nimages in zip(enumerate(x1), x2):
    try:
        label = int(gt_data.loc['/'.join(path.split('/')[-3:-1])]['gt'])
        X1.append(path)
        X2.append(nimages)
        Y.append(label)
    except KeyError:
        continue

x1 = np.array(X1)
x2 = np.array(X2)
y = np.array(Y)

In [ ]:
len(gt_data)

In [ ]:
filter_data_path = '/mnt/storage/ben/data/BeesBook/AdeferDanceDetection/filter_data_rec_mixed.h5'

if not isfile(filter_data_path):
    with h5py.File(filter_data_path, 'w') as f:
        g_X = f.create_group('X')
        g_y = f.create_dataset('y', (len(y), ), np.int64)
        f.create_dataset('n_samples', data=[len(y)])

        for idx in trange(len(y)):
            n_samples = x2[idx]

            samples = []
            for i in range(n_samples):
                image_path = x1[idx] + "image_" + str(i).zfill(3) + ".png"
                samples.append(rgb2gray(io.imread(image_path)))

            samples = np.swapaxes(np.swapaxes(np.array(samples), 0, 2), 0,1)[np.newaxis]
            g_X.create_dataset('{}'.format(idx), data=samples.astype(np.float32))
            g_y[idx] = y[idx] 

In [ ]:
with h5py.File(filter_data_path, 'r') as f:
    n_samples = f['n_samples'][0]
    g_X = f['X']
    seq_lens = []
    
    for idx in trange(n_samples):
        seq_lens.append(g_X['{}'.format(idx)].shape[-1])

In [ ]:
plt.hist(seq_lens)
print(np.max(seq_lens))
print(np.min(seq_lens))
print(np.median(seq_lens))

In [ ]:
indices = list(range(len(seq_lens)))
np.random.shuffle(indices)
train_indices = indices[:int(.8 * len(indices))]
val_indices = indices[int(.8 * len(indices)):]

In [ ]:
def data_gen(data, indices, batch_size, seq_len=128, augment=False, random_border_crop=4, cutoff=10):
    assert(random_border_crop % 2 == 0)
    X = data['X']
    y = data['y']
    width = X['0'].shape[1]
    heigth = X['0'].shape[2]
    
    while True:
        batch_indices = np.random.choice(indices, replace=False, size=batch_size)
        batch_samples = []
        batch_labels = []
        
        for idx in batch_indices:
            sample = X['{}'.format(idx)][:]
            sample = sample[:, :, :, cutoff:-cutoff]
            assert(sample.shape[-1] > 10)
            if sample.shape[-1] > seq_len:
                start_idx = np.random.randint(0, sample.shape[-1] - seq_len)
                sample = sample[:, :, :, start_idx:start_idx+seq_len]
            batch_samples.append(sample)
            batch_labels.append(y[idx])
            
        X_b = np.zeros((batch_size, 1, width, heigth, seq_len), dtype=np.float32)
        for idx, sample in enumerate(batch_samples):
            X_b[idx, :, :, :, :sample.shape[-1]] = sample
            
        if augment:
            crops = np.random.randint(0, random_border_crop, size=(X_b.shape[0], 2))
            crops = [(slice(None, None, None),
                      slice(crops[i, 0], -random_border_crop+crops[i, 0], None),
                      slice(crops[i, 1], -random_border_crop+crops[i, 1], None),
                      slice(None, None, None))
                      for i in range(X_b.shape[0])]
            flips = [(slice(None, None, None),
                      slice(None, None, random.choice([-1, None])),
                      slice(None, None, random.choice([-1, None])),
                      slice(None, None, None))
                      for i in range(X_b.shape[0])]
            X_b = np.array([(seq[crop])[flip] for seq, crop, flip in zip(X_b, crops, flips)])
        else:
            crop = random_border_crop // 2
            X_b = X_b[:, :, crop:-crop, crop:-crop, :]
            
        yield X_b, np.array(batch_labels)

In [ ]:
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import Adam, Nadam
from keras.utils.io_utils import HDF5Matrix
from keras import backend as K
from keras.layers.convolutional import Conv3D, AveragePooling3D
from keras.layers import Input
from keras.models import Model
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf

In [ ]:
class AdamWithWeightnorm(Adam):
    def get_updates(self, params, constraints, loss):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * self.iterations))

        t = self.iterations + 1
        lr_t = lr * K.sqrt(1. - K.pow(self.beta_2, t)) / (1. - K.pow(self.beta_1, t))

        shapes = [K.get_variable_shape(p) for p in params]
        ms = [K.zeros(shape) for shape in shapes]
        vs = [K.zeros(shape) for shape in shapes]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):

            # if a weight tensor (len > 1) use weight normalized parameterization
            # this is the only part changed w.r.t. keras.optimizers.Adam
            ps = K.get_variable_shape(p)
            if len(ps)>1:

                # get weight normalization parameters
                V, V_norm, V_scaler, g_param, grad_g, grad_V = get_weightnorm_params_and_grads(p, g)

                # Adam containers for the 'g' parameter
                V_scaler_shape = K.get_variable_shape(V_scaler)
                m_g = K.zeros(V_scaler_shape)
                v_g = K.zeros(V_scaler_shape)

                # update g parameters
                m_g_t = (self.beta_1 * m_g) + (1. - self.beta_1) * grad_g
                v_g_t = (self.beta_2 * v_g) + (1. - self.beta_2) * K.square(grad_g)
                new_g_param = g_param - lr_t * m_g_t / (K.sqrt(v_g_t) + self.epsilon)
                self.updates.append(K.update(m_g, m_g_t))
                self.updates.append(K.update(v_g, v_g_t))

                # update V parameters
                m_t = (self.beta_1 * m) + (1. - self.beta_1) * grad_V
                v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(grad_V)
                new_V_param = V - lr_t * m_t / (K.sqrt(v_t) + self.epsilon)
                self.updates.append(K.update(m, m_t))
                self.updates.append(K.update(v, v_t))

                # if there are constraints we apply them to V, not W
                if p in constraints:
                    c = constraints[p]
                    new_V_param = c(new_V_param)

                # wn param updates --> W updates
                add_weightnorm_param_updates(self.updates, new_V_param, new_g_param, p, V_scaler)

            else: # do optimization normally
                m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
                v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
                p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

                self.updates.append(K.update(m, m_t))
                self.updates.append(K.update(v, v_t))

                new_p = p_t
                # apply constraints
                if p in constraints:
                    c = constraints[p]
                    new_p = c(new_p)
                self.updates.append(K.update(p, new_p))
        return self.updates


def get_weightnorm_params_and_grads(p, g):
    ps = K.get_variable_shape(p)

    # construct weight scaler: V_scaler = g/||V||
    V_scaler_shape = (ps[-1],)  # assumes we're using tensorflow!
    V_scaler = K.ones(V_scaler_shape)  # init to ones, so effective parameters don't change

    # get V parameters = ||V||/g * W
    norm_axes = [i for i in range(len(ps) - 1)]
    V = p / tf.reshape(V_scaler, [1] * len(norm_axes) + [-1])

    # split V_scaler into ||V|| and g parameters
    V_norm = tf.sqrt(tf.reduce_sum(tf.square(V), norm_axes))
    g_param = V_scaler * V_norm

    # get grad in V,g parameters
    grad_g = tf.reduce_sum(g * V, norm_axes) / V_norm
    grad_V = tf.reshape(V_scaler, [1] * len(norm_axes) + [-1]) * \
             (g - tf.reshape(grad_g / V_norm, [1] * len(norm_axes) + [-1]) * V)

    return V, V_norm, V_scaler, g_param, grad_g, grad_V


def add_weightnorm_param_updates(updates, new_V_param, new_g_param, W, V_scaler):
    ps = K.get_variable_shape(new_V_param)
    norm_axes = [i for i in range(len(ps) - 1)]

    # update W and V_scaler
    new_V_norm = tf.sqrt(tf.reduce_sum(tf.square(new_V_param), norm_axes))
    new_V_scaler = new_g_param / new_V_norm
    new_W = tf.reshape(new_V_scaler, [1] * len(norm_axes) + [-1]) * new_V_param
    updates.append(K.update(W, new_W))
    updates.append(K.update(V_scaler, new_V_scaler))


# data based initialization for a given Keras model
def data_based_init(model, input):
    # input can be dict, numpy array, or list of numpy arrays
    if type(input) is dict:
        feed_dict = input
    elif type(input) is list:
        feed_dict = {tf_inp: np_inp for tf_inp,np_inp in zip(model.inputs,input)}
    else:
        feed_dict = {model.inputs[0]: input}

    # add learning phase if required
    if model.uses_learning_phase and K.learning_phase() not in feed_dict:
        feed_dict.update({K.learning_phase(): 1})

    # get all layer name, output, weight, bias tuples
    layer_output_weight_bias = []
    for l in model.layers:
        if hasattr(l, 'W') and hasattr(l, 'b'):
            assert(l.built)
            layer_output_weight_bias.append( (l.name,l.get_output_at(0),l.W,l.b) ) # if more than one node, only use the first

    # iterate over our list and do data dependent init
    sess = K.get_session()
    for l,o,W,b in layer_output_weight_bias:
        print('Performing data dependent initialization for layer ' + l)
        m,v = tf.nn.moments(o, [i for i in range(len(o.get_shape())-1)])
        s = tf.sqrt(v + 1e-10)
        updates = tf.group(W.assign(W/tf.reshape(s,[1]*(len(W.get_shape())-1)+[-1])), b.assign((b-m)/s))
        sess.run(updates, feed_dict)

In [ ]:
def selu(x):
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
        x: A tensor or variable to compute the activation function for.
    # References
        - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * K.elu(x, alpha)

In [ ]:
seq_len = 128

inputs = Input((1, 46, 46, seq_len))

n_filters = 32
l2 = 1e-8

x = Permute((1, 4, 2, 3))(inputs)

x = Conv3D(n_filters, 5, strides=(2, 2, 2), data_format='channels_first', activation=selu,
           kernel_regularizer=regularizers.l2(l2), bias_regularizer=regularizers.l2(l2))(x)
x = MaxPooling3D((1, 2, 2), data_format='channels_first')(x)

x = Conv3D(n_filters * 2, 3, strides=(1, 1, 1), data_format='channels_first', activation=selu,
           kernel_regularizer=regularizers.l2(l2), bias_regularizer=regularizers.l2(l2))(x)
x = MaxPooling3D((2, 2, 2), data_format='channels_first')(x)

x = Conv3D(n_filters * 4, 3, strides=(1, 1, 1), data_format='channels_first', activation=selu,
           kernel_regularizer=regularizers.l2(l2), bias_regularizer=regularizers.l2(l2))(x)

x = AveragePooling3D((28, 2, 2), data_format='channels_first')(x)
x = Flatten()(x)

x = Dropout(.2)(x)
x = Dense(n_filters * 4, activation=selu,
          kernel_regularizer=regularizers.l2(l2), bias_regularizer=regularizers.l2(l2))(x)

x = Dense(3, activation='softmax')(x)

model = Model(inputs, x)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=AdamWithWeightnorm(0.0005), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
base_path = '/mnt/storage/ben/data/BeesBook/AdeferDanceDetection/'

In [ ]:
def schedule(epoch):
    return 0.0005 if epoch < 100 else 0.0005 * .25

scheduler = LearningRateScheduler(schedule)

batch_size = 32
train_gen = data_gen(h5py.File(filter_data_path, 'r'), train_indices, batch_size, augment=True)
val_gen = data_gen(h5py.File(filter_data_path, 'r'), val_indices, batch_size)

hist = model.fit_generator(train_gen, len(train_indices) // batch_size, 200, 
                           validation_data=val_gen, validation_steps=len(val_indices) // batch_size,
                           callbacks=[scheduler])